# Demonstrate Seq2Seq Wrapper with twitter chat log

In [1]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

In [2]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [3]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [4]:
import seq2seq_wrapper

In [6]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/home/suriya/_/tf/tf-seq2seq-wrapper/seq2seq_wrapper.py'>

In [5]:
# esto se tarda a lo más 1 minuto
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [6]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [0]:
# comment the following line if it's the first time training locally
sess = model.restore_last_session()

In [9]:
# éste es el entrenamiento, déjalo correr lo más que puedas y manda screenshots de lo que vaya
# apareciendo como output
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>

Model saved to disk at iteration #500
val   loss : 3.370399
Interrupted by user at iteration 565


In [7]:
sess = model.restore_last_session()

In [10]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(256, 20)


In [2]:
import os
import re
root_name = 'tb12_goat'
count = 0
regex = re.compile(root_name)
for f in os.listdir():
    match = regex.match(f)
    if match:
        if int(match.end()) > count:
            count = int(match.end())
count += 1
conversation = ''

In [1]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            chat = 'q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded))
            print(chat)
            conversation.append('{}\n'.format(chat))
            replies.append(decoded)
filename = '{}{}.txt'.format(root_name, str(count))
with open(filename, 'x') as f:
    f.write(conversation)
    count += 1
    conversation = ''

NameError: name 'input_' is not defined